In [ ]:
# import tensorflow as tf
import tensorflow.compat.v1 as tf

In [ ]:
# Create TensorFlow object called tensor
# tf.compat.v1.disable_eager_execution()
hello_constant = tf.constant('Hello World!')
with tf.Session() as sess:
    # Run the tf.constant operation in the session
    output = sess.run(hello_constant)
    print(output)

In [ ]:
# 此函数可以理解为形参，用于定义过程，在执行的时候再赋具体的值
x = tf.placeholder(tf.string)
y = tf.placeholder(tf.int32)
z = tf.placeholder(tf.float32)
with tf.Session() as sess:
    output = sess.run(x, feed_dict={x: 'Test String', y: 123, z: 45.67})
    print(output)

In [ ]:
x = tf.constant(10)
x = tf.Variable(5)
x = tf.add(5, 2)  # 7
x = tf.subtract(10, 4) # 6
y = tf.multiply(2, 5)  # 10
z = tf.divide(10, 2) # 5
tf.matmul(a, b) # matrix multiplication, the order of a and b matters!
# cast a float to integer
tf.subtract(tf.cast(tf.constant(2.0), tf.int32), tf.constant(1))   # 1

In [ ]:
from tensorflow.examples.tutorials.mnist import input_data
from test import *

def mnist_features_labels(n_labels):
    """
    Gets the first <n> labels from the MNIST dataset
    :param n_labels: Number of labels to use
    :return: Tuple of feature list and label list
    """
    mnist_features = []
    mnist_labels = []

    mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True)

    # In order to make quizzes run faster, we're only looking at 10000 images
    for mnist_feature, mnist_label in zip(*mnist.train.next_batch(10000)):

        # Add features and labels if it's for the first <n>th labels
        if mnist_label[:n_labels].any():
            mnist_features.append(mnist_feature)
            mnist_labels.append(mnist_label[:n_labels])

    return mnist_features, mnist_labels

In [ ]:
def print_epoch_stats(epoch_i, sess, last_features, last_labels):
    """
    Print cost and validation accuracy of an epoch
    """
    current_cost = sess.run(
        cost,
        feed_dict={features: last_features, labels: last_labels})
    valid_accuracy = sess.run(
        accuracy,
        feed_dict={features: valid_features, labels: valid_labels})
    print('Epoch: {:<4} - Cost: {:<8.3} Valid Accuracy: {:<5.3}'.format(
        epoch_i,
        current_cost,
        valid_accuracy))

In [ ]:
import math
def batches(batch_size, features, labels):
    """
    Create batches of features and labels
    :param batch_size: The batch size
    :param features: List of features
    :param labels: List of labels
    :return: Batches of (Features, Labels)
    """
    assert len(features) == len(labels)

    output_batches = []
    
    sample_size = len(features)
    for start_i in range(0, sample_size, batch_size):
        end_i = start_i + batch_size
        batch = [features[start_i:end_i], labels[start_i:end_i]]
        output_batches.append(batch)
        
    return output_batches

In [ ]:
# Number of features (28*28 image is 784 features)
n_features = 784
# Number of labels/classes
n_classes = 10

# Features and Labels
# Single-precision floating-point format, usually occupying 32 bits (4 bytes) in computer memory
# None for variable batch sizes, because the last batch may be smaller.
features = tf.placeholder(tf.float32, [None, n_input])
labels = tf.placeholder(tf.float32, [None, n_classes])

# Weights and Biases
# use the tf.truncated_normal() function to generate random numbers from a normal distribution.
# goal is to get 0 mean and small equal variance.
weights = tf.Variable(tf.truncated_normal((n_features, n_labels)))
bias = tf.Variable(tf.zeros(n_labels))

# Linear Function xW + b
logits = tf.add(tf.matmul(features, weights), bias)

# Training set.
### train_features, train_labels = mnist_features_labels(n_labels)
# Import MNIST data
mnist = input_data.read_data_sets('/datasets/ud730/mnist', one_hot=True, reshape=False)

# The features are already scaled and the data is shuffled
train_features = mnist.train.images
valid_features = mnist.validation.images
test_features = mnist.test.images

train_labels = mnist.train.labels.astype(np.float32)
valid_labels = mnist.validation.labels.astype(np.float32)
test_labels = mnist.test.labels.astype(np.float32)

# Define loss and optimizer
learning_rate = tf.placeholder(tf.float32)
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=labels))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
### prediction = tf.nn.softmax(logits)
# Cross entropy. Output is one-hot encoding.
### cross_entropy = -tf.reduce_sum(labels * tf.log(prediction), reduction_indices=1)
# Average cross entropy over the entire training set
### cost = tf.reduce_mean(cross_entropy)

# Calculate accuracy
# Reducing loss/cost means increasing accuracy
correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [ ]:
init = tf.global_variables_initializer()

batch_size = 128
epochs = 10
learn_rate = 0.001

with tf.Session() as sess:
    sess.run(init)

    # Training cycle
    for epoch_i in range(epochs):

        # Loop over all batches
        for batch_features, batch_labels in batches(batch_size, train_features, train_labels):
            train_feed_dict = {
                features: batch_features,
                labels: batch_labels,
                learning_rate: learn_rate}
            sess.run(optimizer, feed_dict=train_feed_dict)

        # Print cost and validation accuracy of an epoch
        print_epoch_stats(epoch_i, sess, batch_features, batch_labels)

    # Calculate accuracy for test dataset
    test_accuracy = sess.run(
        accuracy,
        feed_dict={features: test_features, labels: test_labels})

print('Test Accuracy: {}'.format(test_accuracy))

In [ ]:
### --- Two hidden layers! --- ###
import tensorflow.compat.v1 as tf
from tensorflow.examples.tutorials.mnist import input_data

# Remove previous Tensors and Operations
tf.reset_default_graph()

# The file path to save the data. The ".ckpt" extension stands for "checkpoint".
save_file = './model.ckpt'
# Class used to save and/or restore Tensor Variables
saver = tf.train.Saver()

mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

# Parameters
learning_rate = 0.001
training_epochs = 20
batch_size = 128  # Decrease batch size if you don't have enough memory
display_step = 1

n_input = 784  # MNIST data input (img shape: 28*28)
n_classes = 10  # MNIST total classes (0-9 digits)

# Graph input
x = tf.placeholder("float", [None, 28, 28, 1]) # features
y = tf.placeholder("float", [None, n_classes]) # labels
# reshapes the 28px by 28px matrices in x into row vectors of 784px.
x_flat = tf.reshape(x, [-1, n_input])

# Size/width of the hidden layer
n_hidden_layer = 256
# Store layers weight & bias
weights = {
    'hidden_layer': tf.Variable(tf.random_normal([n_input, n_hidden_layer]), name='weights_0'),
    'out': tf.Variable(tf.random_normal([n_hidden_layer, n_classes]), name='bias_0')
}
biases = {
    'hidden_layer': tf.Variable(tf.random_normal([n_hidden_layer])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Dropout. Probability to keep units
# During training, a good starting value for keep_prob is 0.5.
# During testing, use a keep_prob value of 1.0 to keep all units and maximize the power of the model.
keep_prob = tf.placeholder(tf.float32)
# Hidden layer with RELU activation function and dropout
layer_1 = tf.add(tf.matmul(x_flat, weights['hidden_layer']), biases['hidden_layer'])
layer_1 = tf.nn.relu(layer_1)
layer_1 = tf.nn.dropout(layer_1, keep_prob)
# Output layer with linear activation function
logits = tf.add(tf.matmul(layer_1, weights['out']), biases['out'])

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)

# Calculate accuracy
# Reducing loss/cost means increasing accuracy
correct_prediction = tf.equal(tf.argmax(logits=logits, 1), tf.argmax(labels=y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    # Load the weights and bias. Don't need to call tf.global_variables_initializer()
    ### saver.restore(sess, save_file)
    
    sess.run(init)
    
    # Training cycle
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob = 0.5})
    
        # Print status for every 10 epochs
        if epoch % 10 == 0:
            valid_accuracy = sess.run(accuracy, feed_dict={features: mnist.validation.images, labels: mnist.validation.labels, keep_prob = 1.0})
            print('Epoch {:<3} - Validation Accuracy: {}'.format(epoch, valid_accuracy))
    # Save the model
    saver.save(sess, save_file)
    print('Trained Model Saved.')

In [ ]:
### --- CNN --- ###
import tensorflow.compat.v1 as tf
from tensorflow.examples.tutorials.mnist import input_data

# Remove previous Tensors and Operations
tf.reset_default_graph()

# The file path to save the data. The ".ckpt" extension stands for "checkpoint".
save_file = './model.ckpt'
# Class used to save and/or restore Tensor Variables
saver = tf.train.Saver()

mnist = input_data.read_data_sets(".", one_hot=True, reshape=False)

# Parameters
learning_rate = 0.00001
training_epochs = 20
batch_size = 128  # Decrease batch size if you don't have enough memory

# Number of samples to calculate validation and accuracy
# Decrease this if you're running out of memory to calculate accuracy
test_valid_size = 256

n_classes = 10  # MNIST total classes (0-9 digits)

# Image Properties
image_width = 10
image_height = 10
color_channels = 1 # input depth

# Output depth
k_output1 = 32
k_output2 = 64
k_output3 = 1024

# Convolution filter
filter_size_width = 5
filter_size_height = 5

# Dropout. Probability to keep units
dropout = 0.75
keep_prob = tf.placeholder(tf.float32)

# Graph input
x = tf.placeholder("float", [None, 28, 28, 1]) # features
y = tf.placeholder("float", [None, n_classes]) # labels
# input = tf.placeholder(tf.float32, shape=[None, image_height, image_width, color_channels])

# Weight and bias for only 1 convolutional layer
# filter_weight = tf.Variable(tf.truncated_normal([filter_size_height, filter_size_width, color_channels, k_output]))
# filter_bias = tf.Variable(tf.zeros(k_output))

# Store layers weight & bias for multiple layers
weights = {
    'wc1': tf.Variable(tf.random_normal([filter_size_height, filter_size_width, color_channels, k_output1])),
    'wc2': tf.Variable(tf.random_normal([filter_size_height, filter_size_width, k_output1, k_output2])),
    'wd1': tf.Variable(tf.random_normal([7*7*k_output2, k_output3])),
    'out': tf.Variable(tf.random_normal([k_output3, n_classes]))}

biases = {
    'bc1': tf.Variable(tf.random_normal([k_output1])),
    'bc2': tf.Variable(tf.random_normal([k_output2])),
    'bd1': tf.Variable(tf.random_normal([k_output3])),
    'out': tf.Variable(tf.random_normal([n_classes]))}

# the first element in this array indicates the stride for batch and last element indicates stride for features. 
# It's good practice to remove the batches or features you want to skip from the data set rather than use a stride to skip them.
strides = [1, 2, 2, 1] # (batch, height, width, depth)

# SAME Padding, the output height and width are computed as:
# out_height = ceil(float(in_height) / float(strides[1]))
# out_width = ceil(float(in_width) / float(strides[2]))
# VALID Padding, the output height and width are computed as:
# out_height = ceil(float(in_height - filter_height + 1) / float(strides[1]))
# out_width = ceil(float(in_width - filter_width + 1) / float(strides[2]))
# padding = 'SAME'

# Convolution
def conv2d(x, W, b, stride=1):
    x = tf.nn.conv2d(x, W, strides=[1, stride, stride, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)

# Apply Max Pooling. 2x2 filters with a stride of 2x2 are common in practice.
def maxpool2d(x, stride=2):
    return tf.nn.max_pool(x, ksize=[1, stride, stride, 1], strides=[1, stride, stride, 1], padding='SAME')

# Layer 1 - 28*28*1 to 28*28*32 to 14*14*32
conv1 = conv2d(x, weights['wc1'], biases['bc1'])
conv1 = maxpool2d(conv1)
# Layer 2 - 14*14*32 to 14*14*64 to 7*7*64
conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
conv2 = maxpool2d(conv2)

# Fully connected layer - 7*7*64 to 1024
fc1 = tf.reshape(conv2, [-1, weights['wd1'].get_shape().as_list()[0]]) # reshape the 7*7*64 matrices into row vectors of 1024 px.
# Or use flatten.
# from tensorflow.contrib.layers import flatten
# fc1 = tf.contrib.layers.flatten(conv2)
fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
fc1 = tf.nn.relu(fc1)
fc1 = tf.nn.dropout(fc1, dropout)

# Output Layer - class prediction - 1024 to 10
logits = tf.add(tf.matmul(fc1, weights['out']), biases['out'])

# Define loss and optimizer
cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))
optimizer = tf.train.GradientDescentOptimizer(learning_rate=learning_rate).minimize(cost)
# Adam optimizer
# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

# Calculate accuracy
# Reducing loss/cost means increasing accuracy
correct_prediction = tf.equal(tf.argmax(logits=logits, 1), tf.argmax(labels=y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

# Launch the graph
with tf.Session() as sess:
    # Load the weights and bias. Don't need to call tf.global_variables_initializer()
    ### saver.restore(sess, save_file)
    
    sess.run(init)
    
    # Training cycle
    for epoch in range(training_epochs):
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            # Run optimization op (backprop) and cost op (to get loss value)
            sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
    
        # Print status for every 10 epochs
        if epoch % 10 == 0:
            # loss = sess.run(cost, feed_dict={x: batch_x, y: batch_y, keep_prob: 1.})
            valid_accuracy = sess.run(accuracy, feed_dict={features: mnist.validation.images[:test_valid_size], labels: mnist.validation.labels[:test_valid_size], keep_prob = 1.0})
            print('Epoch {:<3} - Validation Accuracy: {}'.format(epoch, valid_accuracy))
            # print('Epoch {:>2}, Batch {:>3} - Loss: {:>10.4f} Validation Accuracy: {:.6f}'.format(epoch + 1, batch + 1, loss, valid_accuracy))
    
    # Calculate Test Accuracy
    test_accuracy = sess.run(accuracy, feed_dict={x: mnist.test.images[:test_valid_size], y: mnist.test.labels[:test_valid_size], keep_prob: 1.})
    print('Testing Accuracy: {}'.format(test_accuracy))
    
    # Save the model
    saver.save(sess, save_file)
    print('Trained Model Saved.')

In [ ]:
### Keras ###
# JUPYTER workspaces, using python 3.5, Tensorflow 1.3, and Keras 2.09.
# http://faroit.com/keras-docs/2.0.9/applications/

In [ ]:
from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation, Flatten, Dropout
from keras.layers import Conv2D, MaxPooling2D, Lambda, Cropping2D
from keras.optimizers import RMSprop

batch_size = 128
epochs = 20
num_classes = 5
input_shape = (32,32,3)

model = Sequential()
model.add(Dense(512, activation='relu', input_shape=(784,)))
model.add(Dropout(0.2))
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(num_classes, activation='softmax'))
model.summary()

model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

#####################

# preprocess data
X_normalized = np.array(X_train / 255.0 - 0.5 ) # or use model.add(Lambda)

from sklearn.preprocessing import LabelBinarizer
label_binarizer = LabelBinarizer()
y_one_hot = label_binarizer.fit_transform(y_train)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25)) # dropout=0.25 in Keras equals keep_prob=0.75 in Tensorflow
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
history = model.fit(X_normalized, y_one_hot, batch_size=batch_size, epochs=20, verbose=1, callbacks=[checkpoint, stopper], validation_split=0.2, 
                    shuffle=True)

#############################

from keras.applications.vgg16 import VGG16, decode_predictions
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
import numpy as np

img_path = 'your_image.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

# Take over the original weights. Do not include the top (last) fully connected layer, but use an own layer for your output
model = VGG16(weights='imagenet', include_top=False)

# Perform inference on our pre-processed image
predictions = model.predict(x)

# Check the top 3 predictions of the model
print('Predicted:', decode_predictions(predictions, top=3)[0])


###
from keras.applications.resnet50 import ResNet50

img = image.load_img(img_path, target_size=(224, 224))
model = ResNet50(weights='imagenet', include_top=False)

###
# Whole-model saving (configuration + weights)
from tensorflow.keras.models import load_model

model.save('model.h5')  # creates a HDF5 file 'my_model.h5'
del model  # deletes the existing model

# returns a compiled model identical to the previous one
model = load_model('model.h5')

In [ ]:
### InceptionV3
from keras.applications.inception_v3 import InceptionV3

img = image.load_img(img_path, target_size=(299, 299))
model = InceptionV3(weights='imagenet', include_top=False)

# We can use smaller than the default 299x299x3 input for InceptionV3
# which will speed up training. Keras v2.0.9 supports down to 139x139x3
input_size = 139
# Using Inception with ImageNet pre-trained weights
inception = InceptionV3(weights='imagenet', include_top=False,
                        input_shape=(input_size,input_size,3))

# Freeze the weights (use the pre-trained weights)
for layer in inception.layers:
    layer.trainable = False

inception.summary()

# Transfer learning for CIFAR-10
from keras.layers import Input, Lambda
import tensorflow as tf
from keras.layers import Dense, GlobalAveragePooling2D

# Makes the input placeholder layer 32x32x3 for CIFAR-10
cifar_input = Input(shape=(32,32,3))

# Re-sizes the input with Kera's Lambda layer & attach to cifar_input
resized_input = Lambda(lambda image: tf.image.resize_images( 
    image, (input_size, input_size)))(cifar_input)

# Feeds the re-sized input into Inception model
inp = inception(resized_input)

## Setting `include_top` to False earlier also removed the GlobalAveragePooling2D layer, but we still want it.
x = GlobalAveragePooling2D()(inp)
# Create two new fully-connected layers
x = Dense(512, activation = 'relu')(x)
predictions = Dense(10, activation = 'softmax')(x)

# Imports the Model API
from keras.models import Model

# Creates the model, assuming your final layer is named "predictions"
model = Model(inputs=cifar_input, outputs=predictions)

# Compile the model
model.compile(optimizer='Adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Check the summary of this new model to confirm the architecture
model.summary()

# Add callbacks
checkpoint = ModelCheckpoint(filepath=save_path, monitor='val_loss', save_best_only=True)
stopper = EarlyStopping(monitor='val_acc', min_delta=0.0003, patience=5)
model.fit(callbacks=[checkpoint, stopper])

# Load data
from sklearn.utils import shuffle
from sklearn.preprocessing import LabelBinarizer
from keras.datasets import cifar10

(X_train, y_train), (X_val, y_val) = cifar10.load_data()

# One-hot encode the labels
label_binarizer = LabelBinarizer()
y_one_hot_train = label_binarizer.fit_transform(y_train)
y_one_hot_val = label_binarizer.fit_transform(y_val)

# Shuffle the training & test data
X_train, y_one_hot_train = shuffle(X_train, y_one_hot_train)
X_val, y_one_hot_val = shuffle(X_val, y_one_hot_val)

# We are only going to use the first 10,000 images for speed reasons
# And only the first 2,000 images from the test set
X_train = X_train[:10000]
y_one_hot_train = y_one_hot_train[:10000]
X_val = X_val[:2000]
y_one_hot_val = y_one_hot_val[:2000]

# Use a generator to pre-process our images for ImageNet
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.inception_v3 import preprocess_input

if preprocess_flag == True:
    datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
    val_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
else:
    datagen = ImageDataGenerator()
    val_datagen = ImageDataGenerator()
    
# Train the model
batch_size = 32
epochs = 5
# Note: we aren't using callbacks here since we only are using 5 epochs to conserve GPU time
model.fit_generator(datagen.flow(X_train, y_one_hot_train, batch_size=batch_size), 
                    steps_per_epoch=len(X_train)/batch_size, epochs=epochs, verbose=1, 
                    validation_data=val_datagen.flow(X_val, y_one_hot_val, batch_size=batch_size),
                    validation_steps=len(X_val)/batch_size)

In [ ]:
python drive.py model.h5
python drive.py model.h5 run1
python video.py run1

from scipy import ndimage
import os
import csv

samples = []
with open('./driving_log.csv') as csvfile:
    reader = csv.reader(csvfile)
    for line in reader:
        samples.append(line)

from sklearn.model_selection import train_test_split
train_samples, validation_samples = train_test_split(samples, test_size=0.2)

import cv2
import numpy as np
import sklearn

def generator(samples, batch_size=32):
    num_samples = len(samples)
    while 1: # Loop forever so the generator never terminates
        shuffle(samples)
        for offset in range(0, num_samples, batch_size):
            batch_samples = samples[offset:offset+batch_size]

            images = []
            angles = []
            for batch_sample in batch_samples:
                name = './IMG/'+batch_sample[0].split('/')[-1]
                center_image = ndimage.imread(name)
                # imgBGR = cv2.imread(name)
                # imgRGB = cv2.cvtColor(imgBGR, cv2.COLOR_BGR2RGB)
                center_angle = float(batch_sample[3])
                images.append(center_image)
                angles.append(center_angle)

            # trim image to only see section with road
            X_train = np.array(images)
            y_train = np.array(angles)
            yield sklearn.utils.shuffle(X_train, y_train)

#             steering_center = float(row[3])

#             # create adjusted steering measurements for the side camera images
#             correction = 0.2 # this is a parameter to tune
#             steering_left = steering_center + correction
#             steering_right = steering_center - correction

#             # read in images from center, left and right cameras
#             path = "..." # fill in the path to your training IMG directory
#             img_center = process_image(np.asarray(Image.open(path + row[0])))
#             img_left = process_image(np.asarray(Image.open(path + row[1])))
#             img_right = process_image(np.asarray(Image.open(path + row[2])))

#             # add images and angles to data set
#             car_images.extend(img_center, img_left, img_right)
#             steering_angles.extend(steering_center, steering_left, steering_right)
            
# augmented_images, augmented_measurements = [],[]
# for image, measurement in zip(images, measurements):
#     augmented_images.append(image)
#     augmented_measurements.append(measurement)
#     augmented_images.append(cv2.flig(image,1)) # image_flipped = np.fliplr(image)
#     augmented_measurements.append(measurement*-1.0)
# X_train = np.array(augmented_images)
# y_train = np.array(augmented_measurements)

# Set our batch size
batch_size=32

# compile and train the model using the generator function
train_generator = generator(train_samples, batch_size=batch_size)
validation_generator = generator(validation_samples, batch_size=batch_size)

ch, row, col = 3, 160, 320  # Trimmed image format

model = Sequential()
# model.add(Lambda(lambda x: x / 255.0 - 0.5, input_shape=(160,320,3)))
model.add(Lambda(lambda x: x/127.5 - 1., input_shape=(ch, row, col), output_shape=(ch, row, col)))
model.add(Cropping2D(cropping = ((70,25),(0,0)))) # crop top 70 and bottom 25 pixels, left 0 and right 0
model.compile(loss='mse',
              optimizer='Adam',
              metrics=['accuracy'])

# Visualization
import matplotlib.pyplot as plt

history_object = model.fit_generator(train_generator, samples_per_epoch = ceil(len(train_samples)/batch_size), validation_data = validation_generator,
    nb_val_samples = ceil(len(validation_samples)/batch_size),
    nb_epoch=5, verbose=1)

### print the keys contained in the history object
print(history_object.history.keys())

### plot the training and validation loss for each epoch
plt.plot(history_object.history['loss'])
plt.plot(history_object.history['val_loss'])
plt.title('model mean squared error loss')
plt.ylabel('mean squared error loss')
plt.xlabel('epoch')
plt.legend(['training set', 'validation set'], loc='upper right')
plt.show()